This guide assumes you have gcloud CLI installed and initialized. If you don't, check https://cloud.google.com/sdk/gcloud.

and docker too

In [ ]:
# if running in Google Colab:
!gcloud auth login

In [ ]:
# colab
project_id = "blablalba"

!gcloud config set project {project_id}

But first, we must [allow authentication for the API](https://cloud.google.com/docs/authentication/provide-credentials-adc#local-dev).

In [ ]:
# if running the notebook locally

# Create your credential file
!gcloud auth application-default login

Files can be stored in GCP by using [buckets](https://cloud.google.com/storage/docs/discover-object-storage-gsutil).

Let's start by creating a Cloud Storage bucket to store our DSE code.

In [1]:
# The name of the bucket must be unique among all GCP servers. If you get an error, try changing `bucket_name` to something else.
bucket_name = "give-me-a-valid-bucket-name-pls-1231223"
region = "us-east1"

!gsutil mb -b on -l {region} gs://{bucket_name}

Creating gs://give-me-a-valid-bucket-name-pls-1231223/...
ServiceException: 409 A Cloud Storage bucket named 'give-me-a-valid-bucket-name-pls-1231223' already exists. Try another name. Bucket names must be globally unique across all Google Cloud projects, including those outside of your organization.


Now, let's copy the code that executes the temp-sense-gen to our bucket.

In [2]:
script = "dse.py"

#!gsutil cp {script} gs://{bucket_name}/{script}

# THIS IS NOT NEEDED, THE BUCKET IS ONLY USED FOR TRAINING ARTIFACTS

Our design space exploration job needs to be run in a Docker container that has OpenFASoC and any needed dependencies already installed. _You can think of a container as a lightweight virtual machine._

GCP only supports running containers that use images hosted inside their platform. So, we're going to [create a repository](https://cloud.google.com/artifact-registry/docs/docker/store-docker-container-images) within Artifact Registry. 

In [3]:
repository_name = "openfasoc-docker-repo"
region = "us-east1"

# Create a new Docker repository in GCP
!gcloud artifacts repositories create {repository_name} --repository-format=docker \
--location={region} --description="Docker repository for OpenFASOC"

# Verify that the repository was created
!gcloud artifacts repositories list

ERROR: (gcloud.artifacts.repositories.create) ALREADY_EXISTS: the repository already exists
Listing items under project valid-spark-366013, across all locations.

                                                                              ARTIFACT_REGISTRY
REPOSITORY             FORMAT  MODE                 DESCRIPTION                      LOCATION  LABELS  ENCRYPTION          CREATE_TIME          UPDATE_TIME          SIZE (MB)
openfasoc-docker-repo  DOCKER  STANDARD_REPOSITORY  Docker repository for OpenFASOC  us-east1          Google-managed key  2022-10-20T12:40:56  2022-10-21T22:23:34  2000.079


Now that the repository is created, we want to push a Docker image to it. Luckily, our team already has a working image we can use, so we'll just transfer it to the repository in Artifact Registry.

For that, we start by pulling the image locally:

In [ ]:
# Set up authentication to Docker repositories in the chosen region
!yes 2>/dev/null | gcloud auth configure-docker {region}-docker.pkg.dev

In [5]:
# Pull the image from the OpenFASoC team
src_image_uri = "luccareinehr/openfasoc-dse:latest"
!docker pull {src_image_uri}

latest: Pulling from luccareinehr/openfasoc-dse
Digest: sha256:20f8ad3a0e23f1b250f165070d46a51137b0b5fa7fb6e3d7b827021e32c3cce3
Status: Downloaded newer image for luccareinehr/openfasoc-dse:latest
docker.io/luccareinehr/openfasoc-dse:latest


In [6]:
# Transfer script to Docker image
!docker create --name temp_container {src_image_uri}
!docker cp {script} temp_container:/
!docker commit --change 'ENTRYPOINT [""]' temp_container {src_image_uri}

273ed5c028fc0e08969f383a0a178574bc14d49fa86a88693d6b638c1826868b
sha256:8b38ed10ac97c02a3ffbcf4891450e8af4d9b802d8e8d951d98397f95dd997f2


Then we tag it with our Docker repository, and push it.

In [7]:
project_id = "valid-spark-366013" # you can get your Project ID in the GCP Console or by running `gcloud projects list` in the command line

# Tag the image with our created repository
repository_image_uri = f"{region}-docker.pkg.dev/{project_id}/{repository_name}/openfasoc-dse-image"
!docker tag {src_image_uri} {repository_image_uri}

# Push the image to Artifact Registry
!docker push {repository_image_uri}

Using default tag: latest
The push refers to repository [us-east1-docker.pkg.dev/valid-spark-366013/openfasoc-docker-repo/openfasoc-dse-image]

d0cf9ea8: Preparing 
5969a40e: Preparing 
ffa2a6f9: Preparing 
8d60478c: Preparing 
ad58f93a: Preparing 
f269488d: Preparing 
2fb6bd3d: Preparing 
fb43a5c5: Preparing 
4d0930dc: Preparing 
5bf1625e: Preparing 
e5b9d662: Preparing 
d1676753: Preparing 
e74f4baa: Preparing 
28409955: Preparing 
5ed3b606: Preparing 
93549a68: Preparing 
277664af: Preparing 
34139513: Preparing 
26fb0ebd: Preparing 
add70712: Preparing 
3c5e4277: Preparing 
d86654e5: Layer already exists 5kBlatest: digest: sha256:ae4c59b3c66e159cf33334fb9889d21360dd2210e00a72ae6d6c6134d413ae5e size: 4968


Now we're ready to configure our Design Space Exploration job.

### Create parameters and metrics specs

To run the DSE, we're going to use a product from GCP called Vertex AI, with its Hyperparameter Tuning tool.

Hyperparameters typically refer to coefficients in a Machine Learning model that don't get optimized in the training phase. That is to say, they're parameters that are usually adjusted manually and change the model's behavior.

Since the tool takes the code it is trying to optimize as a black box, we can parametrize a generator from OpenFASoC, as if the parameters were ML Hyperparameters, and run Hyperparemeter Tuning on it.

An optimization algorithm such as Hyperparameter Tuning requires:
- A number of parameters to optimize, as well as the value ranges accepted
- A number of output metrics to evaluate if the model is improving

Let's start by selecting some parameters from the Temperature Sensor Generator to modify, and an output metric to optimize.

In [8]:
from google.cloud.aiplatform import hyperparameter_tuning as hpt

# Dictionary representing parameters to optimize
# The tool passes your dictionary key as a command line argument to your training job, so they must coincide in the script
# Here, we'll use two integer parameters, so we pass them as hpt.IntegerParameterSpec objects
parameter_spec = {
    'header_starting_row': hpt.IntegerParameterSpec(min=1, max=20, scale='linear'),
    'vin_route_conn': hpt.IntegerParameterSpec(min=1, max=4, scale='linear'),
}

# Dictionary representing metrics to optimize
metric_spec={'power': 'minimize'}

/usr/lib/python3/dist-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 1.1build1 is an invalid version and will not be supported in a future release
  warnings.warn(
/usr/lib/python3/dist-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 0.1.43ubuntu1 is an invalid version and will not be supported in a future release
  warnings.warn(


### Create custom job spec

To configure Hyperparameter Tuning, it's also needed to specify what code should be run to use the specified parameters and generate the specified metric.

In our case, we'll run the Temperature Sensor Generator to optimize its power consumption by varying the placement of header auxiliary cells and the number of connections in the VIN route (from the header cells to the VIN ring). 

In [9]:
from google.cloud import aiplatform

machine_type = 'n1-standard-8'
worker_image = f'{repository_image_uri}:latest'

worker_pool_specs = [{
    'machine_spec': {
        'machine_type': machine_type,
    },
    'replica_count': 1,
    'container_spec': {
        'image_uri': worker_image,
        'command': ['python'],
        'args': [script]
    }
}]

custom_job = aiplatform.CustomJob(
    display_name="tempsense-custom-job",
    worker_pool_specs=worker_pool_specs,
    staging_bucket=f'gs://{bucket_name}'
)

### Run Hyperparameter tuning job

First, we must grant permission to Vertex AI for accessing Artifact Registry repositories:

In [ ]:
import subprocess

# Get unique project number from project name / id by using a regex
# Output needs .strip() because it comes with a line break (\n) at the end
project_number = subprocess.run(f"gcloud projects describe {project_id} | grep -oP \"(?<=projectNumber: ')[0-9]+\"", shell=True, stdout=subprocess.PIPE, text=True).stdout.strip()
print(f"Your project's unique number is: {project_number}")

!gcloud projects add-iam-policy-binding {project_id} --member=serviceAccount:service-{project_number}@gcp-sa-aiplatform.iam.gserviceaccount.com --role=roles/artifactregistry.reader

Then we can start the hyperparameter tuning job! 

In [10]:
from google.cloud import aiplatform

parameters_count = len(parameter_spec.keys())
metrics_count=len(metric_spec.keys()) # you can optimize many metrics simulatenously if you want
max_trial_count = 15
parallel_trial_count = 3

hpt_job = aiplatform.HyperparameterTuningJob(
    display_name="tempsense-tuning-job",
    custom_job=custom_job,
    metric_spec=metric_spec,
    parameter_spec=parameter_spec,
    max_trial_count=max_trial_count,
    parallel_trial_count=parallel_trial_count,
    location=region
)
hpt_job.run()

Creating HyperparameterTuningJob
HyperparameterTuningJob created. Resource name: projects/511288030707/locations/us-east1/hyperparameterTuningJobs/3516992450604302336
To use this HyperparameterTuningJob in another session:
hpt_job = aiplatform.HyperparameterTuningJob.get('projects/511288030707/locations/us-east1/hyperparameterTuningJobs/3516992450604302336')
View HyperparameterTuningJob:
https://console.cloud.google.com/ai/platform/locations/us-east1/training/3516992450604302336?project=511288030707
HyperparameterTuningJob projects/511288030707/locations/us-east1/hyperparameterTuningJobs/3516992450604302336 current state:
JobState.JOB_STATE_PENDING
HyperparameterTuningJob projects/511288030707/locations/us-east1/hyperparameterTuningJobs/3516992450604302336 current state:
JobState.JOB_STATE_RUNNING
HyperparameterTuningJob projects/511288030707/locations/us-east1/hyperparameterTuningJobs/3516992450604302336 current state:
JobState.JOB_STATE_RUNNING
HyperparameterTuningJob projects/511288

### Cleanup (important!)

To avoid incurring charges to your Google Cloud account for the resources used, follow these steps:

In [ ]:
# Delete the created GCP bucket
!gsutil rm -r gs://{bucket_name}

# Delete the created GCP Docker repository
!gcloud artifacts repositories delete {repository_name} --location={region}

# Delete the downloaded Docker image (locally)
!docker image rm {src_image_uri}

# Delete the created Docker container (locally)
!docker rm temp_container

Additional resourcers:
- https://codelabs.developers.google.com/vertex_hyperparameter_tuning
- https://github.com/proppy/rad-lab/blob/tuning/modules/silicon_design/scripts/build/notebooks/examples/digital/serv/tuning.md